In [ ]:
import pandas as pd
import numpy as np 
import yfinance as yf 
import matplotlib.pyplot as plt

In [53]:
data = pd.read_csv('data/yahoo1mo15min.csv')
data=data.drop([0,1])
data['Close'] = data['Close'].astype('float')
data['Open'] = data['Open'].astype('float')
data['Volume'] = data['Volume'].astype('float')
data['High'] = data['High'].astype('float')
data['Low'] = data['Low'].astype('float')
data.rename(columns={'Price':'Time'},inplace=True)
data


,Time,Adj Close,Close,High,Low,Open,Volume
2,2025-01-10 07:15:00+00:00,2704.0,2704.000000,2706.500000,2703.000000,2705.800049,1231.0
3,2025-01-10 07:30:00+00:00,2704.39990234375,2704.399902,2704.800049,2701.699951,2703.899902,1820.0
4,2025-01-10 07:45:00+00:00,2704.89990234375,2704.899902,2707.199951,2704.000000,2704.399902,5410.0
5,2025-01-10 08:00:00+00:00,2705.800048828125,2705.800049,2705.899902,2700.000000,2704.899902,4902.0
6,2025-01-10 08:15:00+00:00,2703.5,2703.500000,2707.100098,2702.100098,2705.800049,4120.0
...,...,...,...,...,...,...,...
1918,2025-02-10 06:00:00+00:00,2928.60009765625,2928.600098,2932.800049,2928.000000,2930.899902,2080.0
1919,2025-02-10 06:15:00+00:00,2928.199951171875,2928.199951,2928.600098,2925.500000,2928.500000,1627.0
1920,2025-02-10 06:30:00+00:00,2931.199951171875,2931.199951,2931.899902,2927.699951,2928.000000,1772.0
1921,2025-02-10 06:45:00+00:00,2931.5,2931.500000,2932.600098,2929.500000,2931.100098,1337.0


In [ ]:


def RSI_Wilder(data, period=14):
    """
    使用 Wilder’s Smoothing Method (RMA) 计算 RSI。

    参数：
    - data: 必须包含 'close' 列的 Pandas DataFrame
    - period: 默认 14

    返回：
    - 在原 DataFrame 中添加一列 'RSI', 并返回
    """
    # 1) 计算收盘价变化 delta
    delta = data['Close'].diff(1)
    
    # 2) 计算涨幅（gain）和跌幅（loss）
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    
    # 3) 使用前 period 条数据，计算初始均值 (简单平均)
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
    # 4) 创建数组来存储 Wilder’s 平滑结果 (RMA)
    rma_gain = np.zeros_like(gain)
    rma_loss = np.zeros_like(loss)
    
    rma_gain[:period] = np.nan
    rma_loss[:period] = np.nan

    # 5) 找到第一个非 NaN 的索引，作为初始值
    first_valid = period
    
    if len(data) >= period:
        rma_gain[first_valid] = avg_gain.iloc[first_valid]
        rma_loss[first_valid] = avg_loss.iloc[first_valid]
        
        # 6) 从第 (period+1) 行开始，Wilder’s 平滑递推
        for i in range(first_valid + 1, len(data)):
            rma_gain[i] = (rma_gain[i-1] * (period - 1) + gain.iloc[i]) / period
            rma_loss[i] = (rma_loss[i-1] * (period - 1) + loss.iloc[i]) / period
    
    # 7) 计算 RSI
    RS = rma_gain / rma_loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    
    # 将 RSI 值写回 DataFrame
    data['RSI'] = RSI
    
    return data


In [ ]:
def Bollinger_Bands(data, window=20, num_std=2, calc_relative_width=False):
    """
    计算布林带 (Bollinger Bands) 及布林带宽度。
    
    参数：
    - data: 包含 'close' 列的 Pandas DataFrame
    - window: 移动平均窗口大小，默认是 20
    - num_std: 标准差倍数，默认是 2
    - calc_relative_width: 是否计算相对宽度, 默认为 False
    
    返回：
    - 带有布林带新增列的 Pandas DataFrame:
      'BB_Middle', 'BB_Upper', 'BB_Lower', 'BB_Width', (可选) 'BB_Ratio'
    """
    # 计算中轨（简单移动平均）
    data['BB_Middle'] = data['Close'].rolling(window=window).mean()
    
    # 计算标准差
    data['BB_Std'] = data['Close'].rolling(window=window).std()
    
    # 计算上轨和下轨
    data['BB_Upper'] = data['BB_Middle'] + num_std * data['BB_Std']
    data['BB_Lower'] = data['BB_Middle'] - num_std * data['BB_Std']
    
    # 计算布林带绝对宽度
    data['BB_Width'] = data['BB_Upper'] - data['BB_Lower']
    
    # 如果需要，计算布林带相对宽度
    if calc_relative_width:
        data['BB_Ratio'] = data['BB_Width'] / data['BB_Middle']
    
    return data


In [56]:
# 订单流不平衡度（需tick数据，可用分钟级近似）
data['imbalance'] = (data['High'] - data['Open']) / (data['Open'] - data['Low'] + 1e-5)

# 波动率加速度
data['vol_acc'] = data['Close'].pct_change().diff().rolling(4).mean()
data['breakout'] = (data['High'] > data['High'].shift(1).rolling(3).max()).astype(int)
fast_period = 12
slow_period = 26
data['macd'] = data['Close'].ewm(span=fast_period).mean() - data['Close'].ewm(span=slow_period).mean()

# 动态支撑阻力
data['pivot'] = (data['High'] + data['Low'] + data['Close']) / 3

In [ ]:
def volume_ratio(data,window = 5):
    data['volume_ratio'] = data['Volume']/data['Volume'].rolling(window=window).mean()
    return data

In [ ]:
data= RSI_Wilder(data)
data= volume_ratio(data)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(data['Time'], data['Close'], label='Price', color='blue')
plt.plot(data['Time'], data['BB_Lower'], label='Lower Band', linestyle='--', color='green')
plt.plot(data['Time'], data['BB_Upper'], label='Upper Band', linestyle='--', color='red')
    

In [ ]:
data['target'] = data['Close'].shift(-1)

# 因为最后一天没有下一天的价格，所以会产生NaN，去掉最后一条数据
data.dropna(inplace=True)

# 构建特征X，这里仅使用当天价格作为特征(极简例子)
X = data[['Close']].values
y = data['target'].values

# 7:3划分训练集和测试集(按时间顺序)
train_size = int(len(data)*0.7)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print("训练集大小:", X_train.shape, y_train.shape)
print("测试集大小:", X_test.shape, y_test.shape)

In [62]:
import requests

# Binance API endpoint for order book data
url = 'https://api.binance.com/api/v3/depth'

# 选择交易对，例：黄金/USDT或BTC/USDT
symbol = 'XAUT'
limit = 10  # 获取前10个买单和卖单

# 发起请求
response = requests.get(url, params={'symbol': symbol, 'limit': limit})

# 打印获取到的Order Book数据
order_book = response.json()
print(order_book)


{'code': -1121, 'msg': 'Invalid symbol.'}
